# GoogleNet

    '''
    参数设置参考Justin 598WI2022课件
    
    5轮训练: 13min 40.6s
    Test Accuracy: 75.98%

    '''

<img src="./image/googlenet1.png" alt="Model Image" width="800">
<img src="./image/googlenet2.png" alt="Model Image" width="800">
<img src="./image/googlenet3.png" alt="Model Image" width="800">

In [1]:
# 环境配置
%cd ../../
import sys
sys.path.append('./python')

d:\sgd-代码库\torch2.0-paly\sgd_deep_learning\sgd_cv


In [2]:
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

from sgd_cv.model import GoogleNet

## 测试模型

In [3]:
# 测试模型结构
model = GoogleNet(num_classes=10)
print(model)

# 随机生成一个批次的输入 (cifar10 图像大小: 3x227x227)
input_tensor = torch.randn(1, 3, 224, 224) # 是否设为224
output = model(input_tensor)

print(f"输入张量大小: {input_tensor.shape}")
print(f"输出张量大小: {output.shape}")  # 应为 [1, 10]


GoogleNet(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (4): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (5): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (inception3): Sequential(
    (0): Inception(
      (branch1): Sequential(
        (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (branch2): Sequential(
        (0): Conv2d(192, 96, kernel_size=(1, 1)

## 训练模型

In [4]:
# 数据集加载
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224), # 224
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225]),
                                ])
# CIFAR10
train_dataset = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义模型、损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GoogleNet(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

Files already downloaded and verified
Files already downloaded and verified
cuda


In [5]:
# 训练
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

100%|██████████| 782/782 [02:41<00:00,  4.83it/s]


Epoch [1/5], Loss: 1.3479


100%|██████████| 782/782 [02:43<00:00,  4.77it/s]


Epoch [2/5], Loss: 0.9927


100%|██████████| 782/782 [02:42<00:00,  4.82it/s]


Epoch [3/5], Loss: 0.8496


100%|██████████| 782/782 [02:47<00:00,  4.68it/s]


Epoch [4/5], Loss: 0.7520


100%|██████████| 782/782 [02:45<00:00,  4.72it/s]

Epoch [5/5], Loss: 0.6744


In [6]:
# 测试
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 75.98%
